In [ ]:
#TEAM VNS HELPERS https://github.com/ArnaudFickinger/astronomical_dataset_generator

#add noise to data, data must be in [x,y,z] format or you know what you are doing
#possible type of noise are gaussian/uniform random noise
def addnoise(data,mu,sigma,typ="gaussian"):
    print("adding ",typ," noise to data......")
    print("mu=",mu," and sigma=",sigma)
    ldata=np.copy(data)
    shape=np.shape(ldata)
    ldata=ldata.flatten()
    size=len(ldata)
    if typ=="gaussian":
        ldata+=np.random.normal(mu,sigma,size=size)
    elif typ=="uniform":        
        ldata+=sigma*(np.random.rand(size)-0.5+mu)
    else:
        print("invalid type of noise!")
        print("available types of noise are gaussian or uniform.")
        exit(1)
    return ldata.reshape(shape)
#randomly delete percentage of data assigned by percent
#you may want to do this to sparsify the data due random conditions like weathers, people in charge of observing ask for a day off or so.
def sparsify(data,percent):
    n=len(data)
    ndrawn=int(n*(1-percent))
    print("sparsify ",n,"data into ",n-ndrawn," data...")
    dellist=random.sample(range(n),ndrawn)
    return np.delete(data,dellist,axis=0)
def find_cycle(data):
  time = []
  max_val = -1* np.inf
  for i in range(len(data)):
    if data['Right Ascension'][i] > max_val:
      max_val = data['Right Ascension'][i]
    elif (max_val - data['Right Ascension'][i]) > 300:
      max_val = data['Right Ascension'][i]
      time.append(data['Time'][i])
  return time
#Fourier feature generating function
def fourier_featurize(X, d = 5, freq = 1):
  data = []
  for i in range(d):
    if i == 0:
      data.append(np.cos(i * freq * X).reshape(1,-1)[0])
    else:
      data.append(np.sin(i * freq * X).reshape(1,-1)[0])
      data.append(np.cos(i * freq * X).reshape(1,-1)[0])
  return np.array(data).T
# estimate_frequency(X.reshape(len(X)), Y_new)

In [ ]:
#helper functions to reset time so that it cycle and function to control the scafcle of the data
def reset_time(Time,cycle):
    New_time = Time - Time.min(axis = 0)
    mod_time = New_time % cycle
    return mod_time
    
def scale(X,max_val):
    n = len(X)
    X_std = (np.array([((X[i] - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))) for i in range(0,n)]))*max_val 
    return X_std

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris, EarthLocation
from astropy.coordinates import get_body_barycentric, get_body, get_moon
from sklearn.preprocessing import PolynomialFeatures
from datetime import datetime, timedelta
from time import mktime

#Helpers
#Cartesian Coordinates relative to earth, conversion implemented from http://faraday.uwyo.edu/~admyers/ASTR5160/handouts/51605.pdf
def convert_cartesian(ra, decl, dist):
  x = dist * np.cos([ra]) * np.cos([decl])
  y = dist * np.sin([ra]) * np.cos([decl])
  z = dist * np.sin([decl])
  return x[0], y[0], z[0]
#Inverse of Cartesian coordinate function, returns cartesian coordinates to ephemeri, RA/DECL/DIST, returns radians or degrees
def convert_ephemeri(x, y , z , radian = True):
  dist = np.sqrt(x**2 + y**2 + z**2)
  ra = np.arctan2(y,x)
  decl = np.arcsin(z/dist)
  #TODO: Finish this
  if radian:
    return ra, decl, dist
  else:
    return math.degrees(ra), math.degrees(decl), dist
#Progress bar helper function
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))
#Data Generator
def generate_ephemeri(body, date_range, earth_loc):
  out = display(progress(0, periods), display_id=True)
  ras = []
  decs = []
  diss = []
  xs = []
  ys = []
  zs = []
  #Generating dates
  loc = EarthLocation.of_site(earth_loc)

  #Generating Ephemeri's
  i = 0
  for t in date_range:
    i += 1
    t = Time(str(t))
    with solar_system_ephemeris.set('jpl'): #Need jplephem library for this
      b = get_body(body, t, loc)
    ra = b.ra.degree
    dec = b.dec.degree
    dis = b.distance.value
    x, y, z = convert_cartesian(b.ra.radian, b.dec.radian, dis)

    ras.append(ra)
    decs.append(dec)
    diss.append(dis)
    xs.append(x)
    ys.append(y)
    zs.append(z)
    #update prog bar    
    out.update(progress(i, periods))
  data = pd.DataFrame({"Time": date_range, "Right Ascension": ras, "Declination": decs, 'Distance': diss, "X": xs, "Y": ys, "Z": zs})
  return data
def find_cycle(data):
  time = []
  max_val = -1* np.inf
  for i in range(len(data)):
    if data['Right Ascension'][i] > max_val:
      max_val = data['Right Ascension'][i]
    elif (max_val - data['Right Ascension'][i]) > 300:
      max_val = data['Right Ascension'][i]
      time.append(data['Time'][i])
  return time

In [ ]:
#DataFrame stack: https://stackoverflow.com/questions/53509623/2d-array-to-two-columns-in-dataframe